In [ ]:
from typing import List, Tuple
from PIL import Image, ImageDraw, ImageOps
import numpy as np
import cv2
import pytesseract
import matplotlib.pyplot as plt

pytesseract.pytesseract.tesseract_cmd = r'/usr/bin/tesseract'

# Segment function updated to use the trained model for character recognition
def segment(
			source: List[List[Image.Image]]
	) -> Tuple[List[List[List[Image.Image]]], List[List[List[Image.Image]]], List[List[List[Tuple[Image.Image, str]]]]]:
	"""
	Segment the license plates into individual characters and visualize bounding boxes.

	:param source: Output of the extract_boxes function
	:return: Segments of license plates, visualizations of detected bounding boxes before and after filtering, and final segments with recognized characters.
	"""
	segments = []
	visualizations = []
	recognized_characters = []

	for img in source:
		img_segments = []
		img_visualizations = []
		img_recognized = []
		for plate in img:
			plate_segments = []
			plate_visualizations = []
			plate_recognized = []
			
			# Resize the plate to a consistent size
			plate = plate.resize((200, 50))
			
			# Convert to grayscale if not already
			if plate.mode != "L":
				gray = plate.convert("L")
			else:
				gray = plate.copy()
			gray = np.array(gray)
			
			# Preprocess using adaptive thresholding
			blurred = cv2.GaussianBlur(gray, (5, 5), 0)
			thresh = cv2.adaptiveThreshold(blurred, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, 11, 2)
			
			# Invert the thresholded image to ensure white text on black background
			thresh = cv2.bitwise_not(thresh)
			
			# Find contours
			contours, hierarchy = cv2.findContours(thresh, cv2.RETR_CCOMP, cv2.CHAIN_APPROX_SIMPLE)
			hierarchy = hierarchy[0]
			
			# Create an initial visualization with all bounding boxes
			original_visualization = plate.convert("RGB")
			draw = ImageDraw.Draw(original_visualization)
			for contour in contours:
				x, y, w, h = cv2.boundingRect(contour)
				draw.rectangle([x, y, x + w, y + h], outline="red", width=1)
			plate_visualizations.append(original_visualization)
			
			# Filter contours to remove obvious noise, but retain characters and possible extra regions
			bounding_boxes = []
			for i, contour in enumerate(contours):
				x, y, w, h = cv2.boundingRect(contour)
				aspect_ratio = w / float(h)
				area = w * h
				parent_idx = hierarchy[i][3]
				child_idx = hierarchy[i][2]
				
				# Use relative area to filter contours after resizing plate to a consistent size
				if 0.1 < aspect_ratio < 2.0 and 70 < area < 800:
					# Check if the contour is not a hole (i.e., not a child of another character)
					# and if it has no children or its children are significantly smaller
					is_valid_character = True
					if parent_idx != -1:
						# If this contour is a child, it might be a hole, so skip it
						continue
					if child_idx != -1:
						# Iterate over all children and check their area
						while child_idx != -1:
							child_area = cv2.contourArea(contours[child_idx])
							if child_area > 0.5 * area:
								# If the child area is significant, this means it might not be a hole
								is_valid_character = False
								break
							child_idx = hierarchy[child_idx][0]  # Get the next child
					
					if is_valid_character:
						bounding_boxes.append((x, y, w, h))
			
			# Create a visualization after initial filtering
			filtered_visualization = plate.convert("RGB")
			draw = ImageDraw.Draw(filtered_visualization)
			for x, y, w, h in bounding_boxes:
				draw.rectangle([x, y, x + w, y + h], outline="green", width=1)
			plate_visualizations.append(filtered_visualization)
			
			# Sort from left to right
			bounding_boxes = sorted(bounding_boxes, key=lambda box: box[0])
			
			# Step 2: Apply the trained model to recognize characters in each segment
			for x, y, w, h in bounding_boxes:
				segment = plate.crop((x, y, x + w, y + h))
				r = w / h
				new_w, new_h = 60, 80
				segment = segment.resize((new_w-20, new_h-20))
				segment_orig = np.array(segment)
				segment = segment.convert("L")

				# binarialize the image
				before = segment.copy()
				_, segment = cv2.threshold(np.array(segment), 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
				segment = Image.fromarray(segment)

				# place the segment on a black background slightly larger than the original
				background = Image.new('L', (new_w, new_h), 255)
				offset = (10, 10)
				background.paste(segment, offset)
				segment = background     

				# Predict with tessaract
				# ocr_result = pytesseract.image_to_data(
				# 	segment,
				# 	config='--psm 10 --oem 3 -c tessedit_char_whitelist=0123456789ABCDEFGHIJKLMNOPQRSTUVWXYZ',
				# 	output_type=pytesseract.Output.DICT
				# )
				# Predict with easyocr
				ocr_result = reader.recognize(
					np.array(segment),
					allowlist="0123456789ABCDEFGHIJKLMNOPQRSTUVWXYZ"
				)
				ocr_result = {
					"text": [res[1] for res in ocr_result],
					"conf": [res[2]*100 for res in ocr_result]
				}
				
				# Filter based on confidence
				# try:
				# 	ocr_result['conf'][0]
				# except:
					# plt.imshow(segment)
					# plt.show()
				confidences = [float(conf) for conf in ocr_result['conf']]
				
				# Filter blue segments (E parts)
				segment_orig = cv2.cvtColor(segment_orig, cv2.COLOR_RGB2HSV)
				lower_blue = np.array([100, 50, 50])
				upper_blue = np.array([140, 255, 255])
				blue_mask = cv2.inRange(segment_orig, lower_blue, upper_blue)
				blue_pixel_count = cv2.countNonZero(blue_mask)
				total_pixel_count = segment_orig.shape[0] * segment_orig.shape[1]
				blue_ratio = blue_pixel_count / total_pixel_count
				is_blue = blue_ratio > 0.5

				# Filter blobs
				eroded = 255 - np.array(segment)
				eroded = cv2.erode(eroded, np.ones((3, 3)), iterations=8)
				is_blob = np.count_nonzero(eroded) / (new_w*new_h) > 0.05

				if confidences and max(confidences) >= -1 and not is_blue and not is_blob:
					recognized_text = ocr_result['text'][np.argmax(confidences)].strip()
					plate_segments.append(segment)
					plate_recognized.append((segment, recognized_text, max(confidences), ocr_result, before))
			
			img_segments.append(plate_segments)
			img_visualizations.append(plate_visualizations)
			img_recognized.append(plate_recognized)
		segments.append(img_segments)
		visualizations.append(img_visualizations)
		recognized_characters.append(img_recognized)
	return segments, visualizations, recognized_characters

segments, visualizations, recognized_characters = segment(crops)

# Visualization script for debugging
def visualize_bounding_boxes(visualizations: List[List[List[Image.Image]]]):
	for img_vis in visualizations:
		for plate_vis in img_vis:
			for vis in plate_vis:
				plt.imshow(vis)
				plt.show()

In [ ]:
segments, visualizations, recognized_characters = segment(crops)

In [ ]:
ground